# Data Science Project 2024

### We will procced according to the KDD Process. 

The following code first load all the .log files in the...

In [1]:
import os
import json
import pandas as pd

# Folder path containing all .log files
folder_path = "/Users/vatsal/Desktop/SimplyTag"  # Replace with your folder path
output_csv_path = "/Users/vatsal/Desktop/SimplyTag/merged_logs.csv"

# Initialize a list to store JSON data from all files
log_data = []

# Process each .log file in the folder
for file_name in os.listdir(folder_path):
    if file_name.endswith(".log"):  # Check if the file is a .log file
        log_file_path = os.path.join(folder_path, file_name)
        print(f"Processing file: {log_file_path}")
        
        # Read the log file line by line
        with open(log_file_path, "r") as file:
            for line in file:
                # Find the JSON part of each line
                json_start = line.find('{')
                if json_start != -1:
                    json_data = line[json_start:].strip()
                    try:
                        # Parse the JSON and append it to the list
                        log_data.append(json.loads(json_data))
                    except json.JSONDecodeError:
                        print(f"Skipping invalid JSON in file {file_name}: {json_data}")

# Convert the combined list of JSON objects to a DataFrame
if log_data:
    df = pd.DataFrame(log_data)

    # Save the DataFrame to a single CSV file
    df.to_csv(output_csv_path, index=False)
    print(f"All log data successfully merged and saved to {output_csv_path}")
else:
    print("No valid log entries found in the provided files.")


Processing file: /Users/vatsal/Desktop/SimplyTag/orgadata-st8.log
Processing file: /Users/vatsal/Desktop/SimplyTag/combined_logs.log


KeyboardInterrupt: 

In [2]:
df = pd.read_csv("/Users/vatsal/Desktop/SimplyTag/merged_logs.csv", low_memory=False)

In [4]:
print(df.columns)
df.describe

Index(['Level', 'Timestamp', 'PID', 'Logger', 'Message', 'Scope',
       'Application', 'State', 'EventId', 'Exception'],
      dtype='object')


<bound method NDFrame.describe of         Level                     Timestamp      PID  \
0        INFO  2024-11-29T23:00:22.7045112Z      668   
1        INFO  2024-11-29T23:00:22.7065968Z      668   
2        INFO  2024-11-29T23:00:22.7104302Z      668   
3        INFO  2024-11-29T23:00:22.7112929Z      668   
4        INFO  2024-11-29T23:00:22.7124052Z      668   
...       ...                           ...      ...   
2144399  INFO  2024-11-19T23:00:03.2857262Z  2151322   
2144400  INFO  2024-11-19T23:00:03.2859194Z  2151322   
2144401  INFO  2024-11-19T23:00:03.2861440Z  2151322   
2144402  INFO  2024-11-19T23:00:03.2863937Z  2151322   
2144403  INFO  2024-11-19T23:00:03.2865924Z  2151322   

                                                    Logger  \
0                 Microsoft.AspNetCore.Hosting.Diagnostics   
1        Microsoft.AspNetCore.HttpLogging.HttpLoggingMi...   
2          Microsoft.AspNetCore.Routing.EndpointMiddleware   
3        Microsoft.AspNetCore.Mvc.Infrastruct

# Step 1: Data Selection:

The following line...

In [14]:
import pandas as pd
import json

# Load the CSV file
df = pd.read_csv("/Users/vatsal/Desktop/SimplyTag/merged_logs.csv", low_memory=False)

# Check a sample entry to understand the structure
print("Sample Entry from 'State':\n", df['State'].iloc[0])

# Function to safely parse JSON
def parse_json_safe(json_str):
    try:
        # Replace single quotes with double quotes for valid JSON
        return json.loads(json_str.replace("'", '"'))
    except (json.JSONDecodeError, TypeError):
        return None

# Parse 'Scope' and 'State' columns
df['Scope'] = df['Scope'].apply(lambda x: parse_json_safe(x) if isinstance(x, str) else None)
df['State'] = df['State'].apply(lambda x: parse_json_safe(x) if isinstance(x, str) else None)

# Initialize a new DataFrame for selected data
df_selected0 = pd.DataFrame()

# Extract specific fields after parsing JSON
df_selected0['Timestamp'] = df['Timestamp']
df_selected0['Trace-id'] = df['Scope'].apply(lambda x: x.get('TraceId') if isinstance(x, dict) else None)
df_selected0['HTTP Status Code'] = df['State'].apply(lambda x: x.get('StatusCode') if isinstance(x, dict) else None)
df_selected0['Path'] = df['State'].apply(lambda x: x.get('Path') if isinstance(x, dict) else None)
df_selected0['User Agent'] = df['State'].apply(lambda x: x.get('User-Agent') if isinstance(x, dict) else None)

# Inspect the extracted data
print("Extracted Data:\n", df_selected0.head())

# Save the cleaned and selected data to a new CSV file
output_csv_path = "/Users/vatsal/Desktop/SimplyTag/selected_data.csv"
df_selected0.to_csv(output_csv_path, index=False)
print(f"Selected data has been saved to {output_csv_path}")


Sample Entry from 'State':
 {'Protocol': 'HTTP/1.1', 'Method': 'GET', 'ContentType': None, 'ContentLength': None, 'Scheme': 'http', 'Host': 'api.owds.org', 'PathBase': '', 'Path': '/api/v2/versions', 'QueryString': '', '{OriginalFormat}': 'Request starting {Protocol} {Method} {Scheme}://{Host}{PathBase}{Path}{QueryString} - {ContentType} {ContentLength}'}
Extracted Data:
                       Timestamp                          Trace-id  \
0  2024-11-29T23:00:22.7045112Z  3dff50eaf246e6cc55173db4092d5187   
1  2024-11-29T23:00:22.7065968Z  3dff50eaf246e6cc55173db4092d5187   
2  2024-11-29T23:00:22.7104302Z  3dff50eaf246e6cc55173db4092d5187   
3  2024-11-29T23:00:22.7112929Z  3dff50eaf246e6cc55173db4092d5187   
4  2024-11-29T23:00:22.7124052Z  3dff50eaf246e6cc55173db4092d5187   

   HTTP Status Code              Path  \
0               NaN              None   
1               NaN  /api/v2/versions   
2               NaN              None   
3               NaN              None   
4    

### Now we need to procced with the new selected data. So we are loading the selected_data file which we have created and then print the new data-frame.

In [59]:
df_selected = pd.read_csv("/Users/vatsal/Desktop/SimplyTag/selected_data.csv", low_memory=False)

In [60]:
df_selected
df_selected[['User Agent']].nunique()

User Agent    188
dtype: int64

In [64]:
print("Rows with non-null 'User Agent':", df_selected['User Agent'].notnull().sum())


Rows with non-null 'User Agent': 179148


In [65]:
print("Rows with non-null 'Trace-id':", df_selected['Trace-id'].notnull().sum())
print("Rows with non-null 'HTTP Status Code':", df_selected['HTTP Status Code'].notnull().sum())
print("Rows with non-null 'Path':", df_selected['Path'].notnull().sum())


Rows with non-null 'Trace-id': 2045561
Rows with non-null 'HTTP Status Code': 312027
Rows with non-null 'Path': 296979


In [66]:
valid_rows = df_selected[
    df_selected['User Agent'].notnull() &
    df_selected['Trace-id'].notnull() &
    df_selected['HTTP Status Code'].notnull() &
    df_selected['Path'].notnull()
]
print("Rows meeting all conditions:", len(valid_rows))


Rows meeting all conditions: 0


In [68]:
# Step 1: Reserve rows where 'User Agent' is not null
df_with_user_agent = df_selected[df_selected['User Agent'].notnull()]
print(f"Rows reserved with non-null 'User Agent': {len(df_with_user_agent)}")

# Step 2: Drop rows with missing values in critical fields from the reserved rows
df_cleaned = df_with_user_agent.dropna(subset=['Trace-id', 'HTTP Status Code', 'Path'])
print(f"Rows after dropping rows with missing values in critical fields: {len(df_cleaned)}")

# Inspect the cleaned data
print("Cleaned data preview:\n", df_cleaned.head())

Rows reserved with non-null 'User Agent': 179148
Rows after dropping rows with missing values in critical fields: 0
Cleaned data preview:
 Empty DataFrame
Columns: [Timestamp, Trace-id, HTTP Status Code, Path, User Agent]
Index: []


,Timestamp,Trace-id,HTTP Status Code,Path,User Agent


# Step 2: Data Preprocessing

The following lines of code...

In [63]:
# Check for missing values
missing_summary = df_selected.isnull().sum()
print("Missing Values Summary:\n", missing_summary)

# Retain rows where 'User Agent' is not null
df_with_user_agent = df_selected[df_selected['User Agent'].notnull()]
df_with_user_agent['User Agent'].nunique()
# Drop rows with missing values in critical fields but retain rows with valid 'User Agent'
df_final = df_with_user_agent.dropna(subset=['Trace-id', 'HTTP Status Code', 'Path']).copy()

print("Data after handling missing values while retaining rows with 'User Agent':\n", df_final.head())
print("Number of unique User Agents:", df_final['User Agent'].nunique())


Missing Values Summary:
 Timestamp                 0
Trace-id              98843
HTTP Status Code    1832377
Path                1847425
User Agent          1965256
dtype: int64
Data after handling missing values while retaining rows with 'User Agent':
 Empty DataFrame
Columns: [Timestamp, Trace-id, HTTP Status Code, Path, User Agent]
Index: []
Number of unique User Agents: 0


In [46]:
# Drop duplicate entries based on critical columns
df_dropduplicate = df_dropped.drop_duplicates(subset=['Timestamp', 'Trace-id', 'HTTP Status Code', 'Path'])

print(f"Data after removing duplicates: {len(df_dropduplicate)} rows")
df_dropduplicate

Data after removing duplicates: 106480 rows


,Timestamp,Trace-id,HTTP Status Code,Path,User Agent
12,2024-11-29T23:00:22.7170405Z,3dff50eaf246e6cc55173db4092d5187,200.0,/api/v2/versions,NaN
28,2024-11-29T23:00:36.5883159Z,bb52744079ab054a8189bf62d8602370,200.0,/api/v2/versions,NaN
62,2024-11-29T23:01:12.6574762Z,02316aed810d686f6c7a5fbe693f10ca,200.0,/api/v2/versions,NaN
75,2024-11-29T23:01:22.8895503Z,005331c774905da73d2d938d1a1fb153,200.0,/api/v2/versions,NaN
88,2024-11-29T23:01:36.7376115Z,83019acd478b0e66cf0e90a932f663de,200.0,/api/v2/versions,NaN
...,...,...,...,...,...
2144327,2024-11-19T22:58:58.9054399Z,8f56736ae00d6d0905149e664675d3e3,200.0,/api/v2/versions,NaN
2144340,2024-11-19T22:59:03.0837242Z,938fb7915bf6d0bbf1f0cd0f0155acf1,200.0,/api/v2/versions,NaN
2144369,2024-11-19T22:59:55.7616546Z,0cfdcd4505ca4f1969017713cc115bff,200.0,/api/v2/versions,NaN
2144390,2024-11-19T22:59:59.0769271Z,c04e0c475d8a6a9e673f00ae3bb74e41,200.0,/api/v2/versions,NaN


In [47]:
# Convert timestamp to datetime format
df_dropduplicate['Timestamp'] = pd.to_datetime(df_dropduplicate['Timestamp'], errors='coerce')

# Remove rows with invalid timestamps
df_dropduplicate = df_dropduplicate.dropna(subset=['Timestamp'])

# Optionally, create new time-related columns
df_dropduplicate['Date'] = df_dropduplicate['Timestamp'].dt.date
df_dropduplicate['Hour'] = df_dropduplicate['Timestamp'].dt.hour
df_dropduplicate['Day_of_week'] = df_dropduplicate['Timestamp'].dt.day_name()

print(df_dropduplicate[['Timestamp', 'Date', 'Hour', 'Day_of_week']].head())


                             Timestamp        Date  Hour Day_of_week
12 2024-11-29 23:00:22.717040500+00:00  2024-11-29    23      Friday
28 2024-11-29 23:00:36.588315900+00:00  2024-11-29    23      Friday
62 2024-11-29 23:01:12.657476200+00:00  2024-11-29    23      Friday
75 2024-11-29 23:01:22.889550300+00:00  2024-11-29    23      Friday
88 2024-11-29 23:01:36.737611500+00:00  2024-11-29    23      Friday


In [48]:
# Normalize user agent strings (basic example)
df_dropduplicate['User_Agent_Normalized'] = df_dropduplicate['User Agent'].str.lower().str.strip()

print("Sample User Agent Normalization:\n", df_dropduplicate[['User Agent', 'User_Agent_Normalized']].head())


Sample User Agent Normalization:
    User Agent User_Agent_Normalized
12        NaN                   NaN
28        NaN                   NaN
62        NaN                   NaN
75        NaN                   NaN
88        NaN                   NaN


In [49]:
# Define valid HTTP status codes
valid_status_codes = [200, 404]

# Filter rows with valid status codes
df_dropduplicate = df_dropduplicate[df_dropduplicate['HTTP Status Code'].isin(valid_status_codes)]

print(f"Data after filtering valid status codes: {len(df_dropduplicate)} rows")


Data after filtering valid status codes: 102767 rows


In [50]:
def categorize_user_agent(user_agent):
    if pd.isnull(user_agent) or user_agent.strip() == "":
        return "Unknown"
    elif "bot" in user_agent.lower() or "spider" in user_agent.lower():
        return "Bot"
    elif "curl" in user_agent.lower() or "wget" in user_agent.lower():
        return "Script"
    elif "mozilla" in user_agent.lower():
        return "Browser"
    else:
        return "Other"

# Apply the categorization using .loc
df_dropduplicate.loc[:, 'user_agent_category'] = df_dropduplicate['User Agent'].apply(categorize_user_agent)

print(df_dropduplicate[['User Agent', 'user_agent_category']].head())



   User Agent user_agent_category
12        NaN             Unknown
28        NaN             Unknown
62        NaN             Unknown
75        NaN             Unknown
88        NaN             Unknown


In [52]:
df_dropduplicate[['User Agent']].nunique()

User Agent    0
dtype: int64